In [1]:
!pip install transformers

In [26]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [27]:
df = pd.read_excel('data.xlsx')
df.head()

,Unnamed: 0,CASE,Q4 ANSWER,Q4 Score,Q9 ANSWER,Q9 Score,Score 1-4 Q4,Ashray Score,Score 1-4 Q9,Ashray Score.1
0,NaN,Case 2a,"Yes, offering the discounts will help increase...",44,"In a normal scenario, the client would pay 100...",44,2,NaN,2.0,2.0
1,NaN,Case 2a,It depends. If they want to promote the use of...,33,If for every $10 increase the possibility of a...,78,2,NaN,4.0,2.0
2,NaN,Case 2a,the introduction is very profitable from Vivit...,0,the increase in premiums corresponds to an inc...,33,1,NaN,2.0,4.0
3,NaN,Case 2a,"To decide whether this is a good idea, I would...",0,The expected cost increase by 150$ per opt-out...,44,1,NaN,2.0,1.0
4,NaN,Case 2a,Ich würde empfehlen das Angebot auch den beste...,6,`-,0,1,NaN,1.0,1.0


In [28]:
q4_df = df[["Q4 ANSWER","Q4 Score","Score 1-4 Q4"]]
#dropping one entry in German
q4_df = q4_df.drop(4)
q4_df.head()

,Q4 ANSWER,Q4 Score,Score 1-4 Q4
0,"Yes, offering the discounts will help increase...",44,2
1,It depends. If they want to promote the use of...,33,2
2,the introduction is very profitable from Vivit...,0,1
3,"To decide whether this is a good idea, I would...",0,1
5,Existing customers might feel neglected if a p...,33,2


In [29]:
q9_df = df[["Q9 ANSWER","Q9 Score","Score 1-4 Q9"]]
q9_df.head()

,Q9 ANSWER,Q9 Score,Score 1-4 Q9
0,"In a normal scenario, the client would pay 100...",44,2.0
1,If for every $10 increase the possibility of a...,78,4.0
2,the increase in premiums corresponds to an inc...,33,2.0
3,The expected cost increase by 150$ per opt-out...,44,2.0
4,`-,0,1.0


In [30]:
q4_df = q4_df.rename(columns = {"Q4 ANSWER":"text","Q4 Score":"score_100","Score 1-4 Q4":"score_4"})
q9_df = q9_df.rename(columns = {"Q9 ANSWER":"text","Q9 Score":"score_100","Score 1-4 Q9":"score_4"})

In [31]:
q4_df = q4_df[q4_df.text != "`-"]
q9_df = q9_df[q9_df.text != "`-"]

In [32]:
print("Number of enteries for Q9 ",q9_df.shape[0])
print("Number of enteries for Q4 ",q4_df.shape[0])

Number of enteries for Q9  196
Number of enteries for Q4  301


In [33]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.snowball import SnowballStemmer
st = SnowballStemmer('english')

[nltk_data] Downloading package stopwords to /Users/uday/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
# function to clean data
def clean_data(df, col, clean_col):

    # change to lower and remove spaces on either side
    #df[clean_col] = df[col].apply(lambda x: x.lower().strip())

    # remove extra spaces in between
    #df[clean_col] = df[clean_col].apply(lambda x: re.sub(' +', ' ', x))

    # remove punctuation
    df[clean_col] = df[col].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

    # remove stopwords and get the stem
    df[clean_col] = df[clean_col].apply(lambda x: ' '.join(st.stem(text) for text in x.split() if text not in stop_words))

    return df

In [35]:
def normalize(df, col):
    # min-max feature scaling
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return df

In [36]:
q4_df = clean_data(q4_df, 'text', 'text')
q9_df = clean_data(q9_df, 'text', 'text')

# q4_df = normalize(q4_df,'score_4')
#q4_df = normalize(q4_df,'score_100')
# q9_df = normalize(q9_df,'score_4')
#q9_df = normalize(q9_df,'score_100')

In [37]:
q9_df["category"] = ['correct' if x>=.5 else 'wrong' for x in list(q9_df['score_100'])]
q4_df["category"] = ['correct' if x>=.5 else 'wrong' for x in list(q4_df['score_100'])]

In [38]:
q4 = q4_df[["text","category"]]
q9 = q9_df[["text","category"]]

In [39]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'wrong':0,
          'correct':1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [40]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [41]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=32)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [42]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0

    predictions =[]
    class_labels = []

    with torch.no_grad():
        for test_input, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            predictions.append(output.argmax(dim=1))
            class_labels.append(test_label)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc

    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

    # Printing reports
    print(classification_report(class_labels, predictions, labels=[0,1])) #classification report from sklearn
    cnf_matrix = confusion_matrix(class_labels, predictions, labels=[0,1])
    plt.imshow(cnf_matrix, cmap=plt.cm.Blues) #plot confusion matrix grid
    threshold = cnf_matrix.max() / 2 #threshold to define text color
    for i in range(cnf_matrix.shape[0]): #print text in grid
        for j in range(cnf_matrix.shape[1]): 
            plt.text(j, i, cnf_matrix[i,j], color="w" if cnf_matrix[i,j] > threshold else 'black')
    tick_marks = np.arange(len(labels)) #define labeling spacing based on number of classes
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.colorbar()
    plt.tight_layout()

In [43]:
np.random.seed(112)
df_train, df_val = np.split(q9.sample(frac=1, random_state=42), 
                                     [int(.7*len(q9))])

print(len(df_train),len(df_val))

137 59


In [44]:
EPOCHS = 10
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:25<00:00, 53.10s/it]


Epochs: 1 | Train Loss:  0.025 | Train Accuracy:  0.387 | Val Loss:  0.022 | Val Accuracy:  0.525


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:13<00:00, 50.67s/it]


Epochs: 2 | Train Loss:  0.025 | Train Accuracy:  0.453 | Val Loss:  0.023 | Val Accuracy:  0.508


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:07<00:00, 49.58s/it]


...

Epochs: 8 | Train Loss:  0.019 | Train Accuracy:  0.818 | Val Loss:  0.022 | Val Accuracy:  0.729


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:28<00:00, 53.75s/it]


Epochs: 9 | Train Loss:  0.019 | Train Accuracy:  0.810 | Val Loss:  0.020 | Val Accuracy:  0.712


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [04:51<00:00, 58.31s/it]


Epochs: 10 | Train Loss:  0.015 | Train Accuracy:  0.825 | Val Loss:  0.022 | Val Accuracy:  0.729


In [45]:
np.random.seed(112)
df_train, df_val = np.split(q4.sample(frac=1, random_state=42), 
                                     [int(.7*len(q4))])

print(len(df_train),len(df_val))

210 91


In [46]:
EPOCHS = 10
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [07:09<00:00, 61.33s/it]


Epochs: 1 | Train Loss:  0.026 | Train Accuracy:  0.248 | Val Loss:  0.024 | Val Accuracy:  0.374


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [07:33<00:00, 64.75s/it]


Epochs: 2 | Train Loss:  0.024 | Train Accuracy:  0.352 | Val Loss:  0.023 | Val Accuracy:  0.440


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [10:43<00:00, 91.94s/it]


...

Epochs: 8 | Train Loss:  0.021 | Train Accuracy:  0.643 | Val Loss:  0.020 | Val Accuracy:  0.582


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [06:41<00:00, 57.35s/it]


Epochs: 9 | Train Loss:  0.019 | Train Accuracy:  0.681 | Val Loss:  0.020 | Val Accuracy:  0.692


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [06:24<00:00, 54.92s/it]


Epochs: 10 | Train Loss:  0.020 | Train Accuracy:  0.686 | Val Loss:  0.020 | Val Accuracy:  0.670
